# AMEX EDA 有意義 ⭐️⭐️⭐️⭐️⭐️

此 EDA 分析數據並提供一些對設計機器學習管道和選擇模型有用的見解。

In [ ]:
import pandas as pd
import numpy as np
import pickle
from matplotlib import pyplot as plt

# 標籤
​​​
我們首先閱讀訓練數據的標籤。 既沒有缺失值，也沒有重複的 customer_ID。 在 458913 個客戶 ID 中，340000 (74%) 個標籤為 0（好客戶，無默認值），119000 個（26%）標籤為 1（壞客戶，默認值）。
​​​
我們知道好的客戶已經被二次抽樣了 20 倍； 這意味著實際上有 680 萬好客戶。 98%的客戶是好的； 2%是壞的。
​​​
**洞察力：**
- 班級不平衡。 建議使用 StratifiedKFold 進行交叉驗證。
- 因為類是不平衡的，準確率將是評估分類器的一個不好的指標。 [競爭指標](https://www.kaggle.com/competitions/amex-default-prediction/discussion/327464) 是 roc 曲線下面積 (auc) 和召回率的混合。# The labels

We start by reading the labels for the training data. There are neither missing values nor duplicated customer_IDs. Of the 458913 customer_IDs, 340000 (74 %) have a label of 0 (good customer, no default) and 119000 (26 %) have a label of 1 (bad customer, default).

We know that the good customers have been subsampled by a factor of 20; this means that in reality there are 6.8 million good customers. 98 % of the customers are good; 2 % are bad.

**Insight:**
- The classes are imbalanced. A StratifiedKFold for cross-validation is recommended.
- Because the classes are imbalanced, accuracy would be a bad metric to evaluate a classifier. The [competition metric](https://www.kaggle.com/competitions/amex-default-prediction/discussion/327464) is a mix of area under the roc curve (auc) and recall.

In [ ]:
train_labels = pd.read_csv('../input/amex-default-prediction/train_labels.csv')
train_labels.head(2)

In [ ]:
# Check for missing data and duplicated customer_IDs
train_labels.isna().any().any(), train_labels.customer_ID.duplicated().any()

In [ ]:
label_stats = pd.DataFrame({'absolute': train_labels.target.value_counts(),
              'relative': train_labels.target.value_counts() / len(train_labels)})
label_stats['absolute upsampled'] =  label_stats.absolute * np.array([20, 1])
label_stats['relative upsampled'] = label_stats['absolute upsampled'] / label_stats['absolute upsampled'].sum()
label_stats

＃ 數據
​​​
本次比賽的數據集規模相當大。 如果您閱讀原始 csv 文件，則數據幾乎無法放入內存。 這就是我們從 @munumbutt 的 [AMEX-Feather-Dataset](https://www.kaggle.com/datasets/munumbutt/amexfeather) 讀取數據的原因。 在這個 [Feather](https://arrow.apache.org/docs/python/feather.html) 文件中，浮點精度已從 64 位降低到 16 位。 並且讀取 Feather 文件比讀取 csv 文件更快，因為 Feather 文件格式是二進制的。
​​​
訓練數據有 550 萬行，測試數據有 1100 萬行。# The data

The dataset of this competition has a considerable size. If you read the original csv files, the data barely fits into memory. That's why we read the data from @munumbutt's [AMEX-Feather-Dataset](https://www.kaggle.com/datasets/munumbutt/amexfeather). In this [Feather](https://arrow.apache.org/docs/python/feather.html) file, the floating point precision has been reduced from 64 bit to 16 bit. And reading a Feather file is faster than reading a csv file because the Feather file format is binary.

There are 5.5 million rows for training and 11 million rows of test data.

In [ ]:
%%time
train = pd.read_feather('../input/amexfeather/train_data.ftr')
test = pd.read_feather('../input/amexfeather/test_data.ftr')
with pd.option_context("display.min_rows", 6):
    display(train)
    display(test)

訓練數據框的目標列對應於 train_labels.csv 的目標列。 在訓練數據的csv文件中，沒有目標列； 為方便起見，它已被加入到 Feather 文件中。

S_2 是報表日期。 所有火車結單日期都在 2017 年 3 月至 2018 年 3 月（13 個月）之間，並且沒有遺漏任何結單日期。 所有測試聲明日期都在 2018 年 4 月和 2019 年 10 月之間。這意味著 train 和 test 的聲明日期不重疊：

In [ ]:
print('Train statement dates: ', train.S_2.min(), train.S_2.max(), train.S_2.isna().any())
print('Test statement dates: ',  test.S_2.min(), test.S_2.max(), test.S_2.isna().any())




**洞察力：**
- 測試數據來自與訓練數據不同的經濟周期階段。 我們的模型無法了解經濟周期的影響。

In [ ]:
print(f'Train data memory usage: {train.memory_usage().sum() / 1e9} GBytes')
print(f'Test data memory usage:  {test.memory_usage().sum() / 1e9} GBytes')


訓練數據佔用 2.2 GB RAM。 測試數據是訓練數據的兩倍。

**洞察力：**
- 有了這麼多數據，我們需要關注內存效率。 避免在內存中保留不必要的數據副本，並避免保留不必要的模型副本！
- 儘管大多數機器學習算法都希望整個訓練數據都在內存中，但我們不需要一次加載所有測試數據。 測試數據可以批量處理。
- 您可能希望將訓練和推理代碼分離到兩個筆記本中，這樣您就不會同時在內存中擁有訓練和測試數據。

info 函數顯示大多數其他特徵都有缺失值：


In [ ]:
train.info(max_cols=200, show_counts=True)

**洞察力：**
- 有很多列缺失值：刪除所有缺失值的列不是明智的策略。
- 有很多行缺失值：刪除所有缺失值的行不是一個明智的策略。
- 許多基於決策樹的算法可以處理缺失值。如果我們選擇這樣的模型，我們不需要更改缺失值。
- 神經網絡和其他估計器無法處理缺失值。如果我們選擇這樣的模型，我們需要估算值。請參閱 [本指南](https://www.kaggle.com/code/parulpandey/a-guide-to-handling-missing-values-in-python) 了解許多插補選項的概述。
- 大多數功能都是 16 位浮點數。原始數據（在 csv 文件中）具有更高的精度。通過將其四捨五入到 16 位精度，會丟失一些信息。為了使這種信息丟失更加明顯：1 到 2 之間的每個 float16 數字都是 1/1024 的倍數。這些數字只有小數點後三位！這個精度足以開始比賽；也許我們將不得不在最後切換到更高的精度。

# 計算每個客戶的報表

現在我們可以計算每個客戶有多少行（信用卡對帳單）。 我們看到 80% 的客戶有 13 條語句； 其他 20% 的客戶有 1 到 12 條語句。

**洞察：**我們的模型將不得不處理每個客戶的可變大小輸入（除非我們簡化我們的生活並且只查看@inversion 建議的最新聲明[此處]（https://www.kaggle. com/competitions/amex-default-prediction/discussion/327094）或所有聲明的平均值）。

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
train_sc = train.customer_ID.value_counts().value_counts().sort_index(ascending=False).rename('Train statements per customer')
ax1.pie(train_sc, labels=train_sc.index)
ax1.set_title(train_sc.name)
test_sc = test.customer_ID.value_counts().value_counts().sort_index(ascending=False).rename('Test statements per customer')
ax2.pie(test_sc, labels=test_sc.index)
ax2.set_title(test_sc.name)
plt.show()

# display(train.customer_ID.value_counts().value_counts().sort_index(ascending=False).rename('Train statements per customer'))
# display(train.customer_ID.value_counts().value_counts().sort_index(ascending=False).rename('Test statements per customer'))


讓我們找出這些客戶何時收到最後一份聲明。 最後一次結單日期的直方圖顯示，每位火車客戶在 2018 年 3 月收到最後一次結單。前四個星期六（3 月 3 日、10 日、17 日、24 日）的結單數量高於平均一天。

測試客戶分為兩部分：其中一半在 2019 年 4 月獲得最後一份聲明，另一半在 2019 年 10 月獲得。正如 [此處討論](https://www.kaggle.com/competitions/amex-default-prediction /discussion/327602)，2019 年 4 月的數據用於公共排行榜，2019 年 10 月的數據用於私人排行榜。

In [ ]:
temp = train.S_2.groupby(train.customer_ID).max()
plt.figure(figsize=(16, 4))
plt.hist(temp, bins=pd.date_range("2018-03-01", "2018-04-01", freq="d"),
         rwidth=0.8, color='#ffd700')
plt.title('When did the train customers get their last statements?', fontsize=20)
plt.xlabel('Last statement date per customer')
plt.ylabel('Count')
plt.gca().set_facecolor('#0057b8')
plt.show()
del temp

temp = test.S_2.groupby(test.customer_ID).max()
plt.figure(figsize=(16, 4))
plt.hist(temp, bins=pd.date_range("2019-04-01", "2019-11-01", freq="d"),
         rwidth=0.74, color='#ffd700')
plt.title('When did the test customers get their last statements?', fontsize=20)
plt.xlabel('Last statement date per customer')
plt.ylabel('Count')
plt.gca().set_facecolor('#0057b8')
plt.show()
del temp

**洞察：** 雖然數據是一種時間序列，但我們無法使用 TimeSeriesSplit 進行交叉驗證，因為所有訓練都發生在同一個月。

對於大多數客戶來說，第一個和最後一個聲明相隔大約一年。 再加上我們通常每個客戶有 13 份對帳單，這表明客戶每個月都會收到一張信用卡對帳單。

In [ ]:
temp = train.S_2.groupby(train.customer_ID).agg(['max', 'min'])
plt.figure(figsize=(16, 3))
plt.hist((temp['max'] - temp['min']).dt.days, bins=400, color='#ffd700')
plt.xlabel('days')
plt.ylabel('count')
plt.title('Number of days between first and last statement of customer (train)', fontsize=20)
plt.gca().set_facecolor('#0057b8')
plt.show()

temp = test.S_2.groupby(test.customer_ID).agg(['max', 'min'])
plt.figure(figsize=(16, 3))
plt.hist((temp['max'] - temp['min']).dt.days, bins=400, color='#ffd700')
plt.xlabel('days')
plt.ylabel('count')
plt.title('Number of days between first and last statement of customer (test)', fontsize=20)
plt.gca().set_facecolor('#0057b8')
plt.show()

如果我們根據它所屬的數據集（訓練、公共 lb 和私有 lb）為每個語句（即訓練或測試行）著色，我們會看到每個數據集涵蓋 13 個月。 訓練和測試不重疊，但公共和私人 lb 時期重疊。

In [ ]:
temp = pd.concat([train[['customer_ID', 'S_2']], test[['customer_ID', 'S_2']]], axis=0)
temp.set_index('customer_ID', inplace=True)
temp['last_month'] = temp.groupby('customer_ID').S_2.max().dt.month

plt.figure(figsize=(16, 4))
plt.hist([temp.S_2[temp.last_month == 3],   # ending 03/18 -> training
          temp.S_2[temp.last_month == 4],   # ending 04/19 -> public lb
          temp.S_2[temp.last_month == 10]], # ending 10/19 -> private lb
         bins=pd.date_range("2017-03-01", "2019-11-01", freq="MS"),
         label=['Training', 'Public leaderboard', 'Private leaderboard'],
         stacked=True)
plt.xticks(pd.date_range("2017-03-01", "2019-11-01", freq="QS"))
plt.xlabel('Statement date')
plt.ylabel('Count')
plt.title('The three datasets', fontsize=20)
plt.legend()
plt.show()

# 分類特徵
​​​
根據【數據描述】（https://www.kaggle.com/competitions/amex-default-prediction/data），有十一個分類特徵。 我們為 target=0 和 target=1 繪製直方圖。 對於有缺失值的十個特徵，缺失值由直方圖最右邊的條形表示。
​​​# The categorical features

According to the [data description](https://www.kaggle.com/competitions/amex-default-prediction/data), there are eleven categorical features. We plot histograms for target=0 and target=1. For the ten features which have missing values, the missing values are represented by the rightmost bar of the histogram.


In [ ]:
cat_features = ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']
plt.figure(figsize=(16, 16))
for i, f in enumerate(cat_features):
    plt.subplot(4, 3, i+1)
    temp = pd.DataFrame(train[f][train.target == 0].value_counts(dropna=False, normalize=True).sort_index().rename('count'))
    temp.index.name = 'value'
    temp.reset_index(inplace=True)
    plt.bar(temp.index, temp['count'], alpha=0.5, label='target=0')
    temp = pd.DataFrame(train[f][train.target == 1].value_counts(dropna=False, normalize=True).sort_index().rename('count'))
    temp.index.name = 'value'
    temp.reset_index(inplace=True)
    plt.bar(temp.index, temp['count'], alpha=0.5, label='target=1')
    plt.xlabel(f)
    plt.ylabel('frequency')
    plt.legend()
    plt.xticks(temp.index, temp.value)
plt.suptitle('Categorical features', fontsize=20, y=0.93)
plt.show()


**洞察力：**
- 每個特徵最多有八個類別（包括一個nan類別）。 One-hot 編碼是可行的。
- target=0 和 target=1 的分佈不同。 這意味著每個特徵都提供了一些關於目標的信息。
​​​


# 二進制特徵

兩個特徵是二進制的：
- B_31 始終為 0 或 1。
- D_87 始終為 1 或缺失。

In [ ]:
bin_features = ['B_31', 'D_87']
plt.figure(figsize=(16, 4))
for i, f in enumerate(bin_features):
    plt.subplot(1, 2, i+1)
    temp = pd.DataFrame(train[f][train.target == 0].value_counts(dropna=False, normalize=True).sort_index().rename('count'))
    temp.index.name = 'value'
    temp.reset_index(inplace=True)
    plt.bar(temp.index, temp['count'], alpha=0.5, label='target=0')
    temp = pd.DataFrame(train[f][train.target == 1].value_counts(dropna=False, normalize=True).sort_index().rename('count'))
    temp.index.name = 'value'
    temp.reset_index(inplace=True)
    plt.bar(temp.index, temp['count'], alpha=0.5, label='target=1')
    plt.xlabel(f)
    plt.ylabel('frequency')
    plt.legend()
    plt.xticks(temp.index, temp.value)
plt.suptitle('Binary features', fontsize=20)
plt.show()


**見解：** 如果您為 D_87 估算缺失值，請不要陷入估算平均值的陷阱 - 該功能將變得無用......

# 連續特徵
​​​
如果我們繪製連續特徵的直方圖，我們會看到它們具有各種分佈：# The continuous features

If we plot histograms of the continuous features, we see that they have all kinds of distributions:

In [ ]:
cont_features = sorted([f for f in train.columns if f not in cat_features + ['customer_ID', 'target', 'S_2']])
# print(cont_features)
ncols = 4
for i, f in enumerate(cont_features):
    if i % ncols == 0: 
        if i > 0: plt.show()
        plt.figure(figsize=(16, 3))
        if i == 0: plt.suptitle('Continuous features', fontsize=20, y=1.02)
    plt.subplot(1, ncols, i % ncols + 1)
    plt.hist(train[f], bins=200)
    plt.xlabel(f)
plt.show()

**洞察：** 左端或右端有空白的直方圖表示數據包含異常值。 我們將不得不處理這些異常值。 但這些數據真的是異常值嗎？ 也許它們是，但它們也可能是罕見事件的合法痕跡。 我們不知道...


未完待續...